In [10]:
import pandas as pd
import pycountry
import plotly.express as px
import plotly.graph_objects as go
from db_conn import get_engine
from dash import Dash, html, dcc


db_conn = get_engine()

In [11]:
df_company = pd.read_sql("select * from dw_db.stock", con=db_conn)

df_geo = (
    df_company.groupby(["country_code_iso", "country", "region"])
    .agg({"price_open": "mean"})
    .reset_index()
)

df_geo.head(5)

,country_code_iso,country,region,price_open
0,AR,Argentina,Americas,3.544519e+03
1,BE,Belgium,Europe,5.900501e+09
2,CA,Canada,Americas,8.431943e+03
3,CH,Switzerland,Europe,5.597742e+09
4,CN,China,Asia,3.009300e+04


In [12]:
def get_iso_alpha(country):
    if type(country) != None:
        country = pycountry.countries.get(alpha_2=country)
        iso_alpha = country.alpha_3

    return iso_alpha


In [13]:

df_geo['iso_alpha'] = df_geo['country_code_iso'].apply(lambda x: get_iso_alpha(x))

In [14]:
df_geo.head(20)

,country_code_iso,country,region,price_open,iso_alpha
0,AR,Argentina,Americas,3.544519e+03,ARG
1,BE,Belgium,Europe,5.900501e+09,BEL
2,CA,Canada,Americas,8.431943e+03,CAN
3,CH,Switzerland,Europe,5.597742e+09,CHE
4,CN,China,Asia,3.009300e+04,CHN
5,DE,Germany,Europe,3.732055e+09,DEU
6,FI,Finland,Europe,4.689441e+09,FIN
7,FR,France,Europe,1.467665e+09,FRA
8,GB,United Kingdom,Europe,2.582325e+09,GBR
9,IL,Israel,Asia,2.218735e+03,ISR


In [7]:
figs = go.Figure(
    data=go.Choropleth(
        locations=df_geo["iso_alpha"],
        z=df_geo["price_open"],
        text=df_geo["country"],
        colorscale="Blues",
        autocolorscale=False,
        reversescale=True,
        marker_line_color="darkgray",
        marker_line_width=0.5,
        colorbar_tickprefix="$",
        colorbar_title="Average Price",
    )
)

figs.update_layout(
    title_text="Stock Market",
    geo=dict(showframe=False, showcoastlines=False, projection_type="equirectangular"),
)

figs.show()

In [9]:
fig = px.scatter_geo(df_geo, locations="iso_alpha", color="region",
                     hover_name="country", size="price_open",
                     scope="world")
fig.show()